In [1]:
#CacheBackedEmbeddings
# Embeddings는 재계산을 피하기 위해 저장되거나 일시적으로 캐시될 수 있습니다.
# Embeddings를 캐싱하는 것은 CacheBackedEmbeddings를 사용하여 수행될 수 있습니다. 
# 캐시 지원 embedder는 embeddings를 키-값 저장소에 캐싱하는 embedder 주변에 래퍼입니다. 텍스트는 해시되고 이 해시는 캐시에서 키로 사용됩니다.
#
# CacheBackedEmbeddings를 초기화하는 주요 지원 방법은 from_bytes_store입니다. 이는 다음 매개변수를 받습니다:
#
# - underlying_embedder: 임베딩을 위해 사용되는 embedder.
# - document_embedding_cache: 문서 임베딩을 캐싱하기 위한 ByteStore 중 하나.
# - namespace: (선택 사항, 기본값은 "") 문서 캐시를 위해 사용되는 네임스페이스. 이 네임스페이스는 다른 캐시와의 충돌을 피하기 위해 사용됩니다. 예를 들어, 사용된 임베딩 모델의 이름으로 설정하십시오.
# 주의: 동일한 텍스트가 다른 임베딩 모델을 사용하여 임베딩될 때 충돌을 피하기 위해 namespace 매개변수를 설정하는 것이 중요합니다.


In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [3]:
# VectorStore 를 위해 faiss 이용하는 예시임
# %pip install --upgrade --quiet  langchain-openai faiss-cpu


Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain.storage import LocalFileStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter


from langchain.embeddings import CacheBackedEmbeddings

# embedding 설정
embedding = OpenAIEmbeddings() # model 인자 지정하지 않으면 기본 text-embedding-ada-0020 모델 이용함.
print(f'*임베딩 모델: {embedding.model}')

# 로컬 파일 저장소 설정
# => cache 폴더에 가보면, text-embedding-ada-002b41bd27f-7cf5-5323-bc34-d315497941a6 식으로 각 임베딩 값이 만들어저 있음.
store = LocalFileStore("./cache/")
#store = InMemoryByteStore()  # 메모리 내 바이트 저장소 생성

# 캐시 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding,
    store,
    namespace=embedding.model, # 네임스페이스로 임베딩 모델명 지정
)

# store에서 키들을 순차적으로 가져옵니다.
#print(list(store.yield_keys()))

from langchain_text_splitters import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter 로 문서를 불러와서 chunk 함.
with open("../data/04.급여규정_16.06.01.txt") as f:
    file = f.read()

print(f'*type:{type(file)}')
print(f'*file:\n{file[:100]}\n')

text_splitter = RecursiveCharacterTextSplitter(
    # 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
    chunk_size=250,
    # 청크 간의 중복되는 문자 수를 설정합니다.
    chunk_overlap=50,
    # 문자열 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자로 정규식을 사용할지 여부를 설정합니다.
    is_separator_regex=False,
)

# text_splitter 이용해서 file 텍스트를 문서단위로 출력
# -> separator를 지정하지 않았으므로, 문서에서 기본값 "\n\n" 로 구분해서 나눔.
documents = text_splitter.create_documents([file])
print(f'*문서 분할 수: {len(documents)}')

for document in documents:
    print(document)
    print(f'--' * 20)


*임베딩 모델: text-embedding-ada-002
*type:<class 'str'>
*file:
급여규정
1. 목적
이 규정은 주식회사 모코엠시스에 근무하는 직원의 급여에 관한 제반사항을 정하여 합리적인 급여관리를 실행함을 목적으로 한다.
2. 적용범위
직원의 급여는 관계법령

*문서 분할 수: 17
page_content='급여규정\n1. 목적\n이 규정은 주식회사 모코엠시스에 근무하는 직원의 급여에 관한 제반사항을 정하여 합리적인 급여관리를 실행함을 목적으로 한다.\n2. 적용범위\n직원의 급여는 관계법령, 기타 별도로 정한 것을 제외하고는 이 규정이 정하는 바에 따른다.\n3. 급여의 구성\n3.1 급여는 기본급, 제수당, 상여금, 퇴직금으로 구분한다.\n3.2 일반적으로 월급여라 함은 기본급과 제수당을 합한 금액을 말한다.\n4. 급여조정'
----------------------------------------
page_content='3.2 일반적으로 월급여라 함은 기본급과 제수당을 합한 금액을 말한다.\n4. 급여조정\n급여의 인상은 년 1회 1월 1일부로 실시함을 원칙으로 한다.\n5. 계산과 지급\n5.1 계산기간\n급여계산기간은 매월 1일부터 당월 말일까지로 한다.\n5.2 일할계산\n5.2.1 급여의 변경이 있는 월의 급여는 일할계산한다.\n5.2.2 일할계산은 월의 대소에 관계없이 월급여의 1/30로 한다.\n5.3 일할일수'
----------------------------------------
page_content='5.2.2 일할계산은 월의 대소에 관계없이 월급여의 1/30로 한다.\n5.3 일할일수\n일할계산에 있어 그 일할일수는 실근로일수를 기준으로 한다.\n5.4 계산단위\n급여계산은 원단위에서 이를 절사한다.\n5.5 급여지급일\n5.5.1 급여지급일은 매월 20일로 한다.\n단, 지급일이 휴일인 경우에는 그 전일로 한다.\n5.5.2 비상재해, 기타 부득이한 사유가 발생시에는 지급일을 변경할 수 있다\n5.6 지

In [16]:
# 코드 실행 시간을 측정합니다.
%time db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

CPU times: user 362 ms, sys: 68.5 ms, total: 430 ms
Wall time: 1.59 s


In [18]:
# 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성
# => **벡터 저장소를 다시 생성하려고 하면, 임베딩을 다시 계산할 필요가 없기 때문에 훨씬 더 빠르게 처리됩니다.
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 9 ms, sys: 9.44 ms, total: 18.4 ms
Wall time: 156 ms
